In [ ]:
! pip install face_recognition

### Lets define the mesonet model - mesonet.py

In [ ]:
from keras.models import Model as KerasModel
from keras.layers import Input, Dense,Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Reshape, Concatenate, LeakyReLU
from keras.optimizers import Adam
import os
import glob
import torch
import cv2
from PIL import Image
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
IMGWIDTH = 256
import face_recognition

In [ ]:
class Classifier:
    def __init__():
        self.model = 0
    
    def predict(self, x):
        return self.model.predict(x)
    
    def fit(self, x, y):
        return self.model.train_on_batch(x, y)
    
    def get_accuracy(self, x, y):
        return self.model.test_on_batch(x, y)
    
    def load(self, path):
        self.model.load_weights(path)

class Meso4(Classifier):
    def __init__(self, learning_rate=0.001):
        self.model = self.init_model()
        optimizer = Adam(lr = learning_rate)
        self.model.compile(optimizer = optimizer, loss='mean_squared_error', metrics=['accuracy'])
    
    def init_model(self):
        x = Input(shape=(IMGWIDTH, IMGWIDTH, 3))
        
        x1 = Conv2D(8, (3,3), padding='same', activation='relu')(x)
        x1 = BatchNormalization()(x1)
        x1 = MaxPooling2D(pool_size=(2,2), padding='same')(x1)
        
        x2 = Conv2D(8,(5,5), padding='same', activation='relu')(x1)
        x2 = BatchNormalization()(x2)
        x2 = MaxPooling2D(pool_size=(2,2), padding='same')(x2)
        
        x3 = Conv2D(16, (5,5), padding='same', activation='relu')(x2)
        x3 = BatchNormalization()(x3)
        x3 = MaxPooling2D(pool_size=(2,2), padding='same')(x3)
        
        x4 = Conv2D(16,(5,5), padding='same', activation='relu')(x3)
        x4 = BatchNormalization()(x4)
        x4 = MaxPooling2D(pool_size=(4,4), padding='same')(x4)
        
        y = Flatten()(x4)
        y = Dropout(0.5)(y)
        y = Dense(16)(y)
        y = LeakyReLU(alpha=0.1)(y)
        y = Dropout(0.5)(y)
        y = Dense(1, activation='sigmoid')(y)
        
        return KerasModel(input=x, outputs=y)

### Getting the trained model weights

In [ ]:
!wget https://github.com/PacktPublishing/Machine-Learning-for-Cybersecurity-Cookbook/raw/master/Chapter04/Deepfake%20Recognition/mesonet_weights/Meso4_DF

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

MesoNet_classifier = Meso4()
MesoNet_classifier.load("Meso4_DF")

In [ ]:

# See github.com/timesler/facenet-pytorch:
#from facenet_pytorch import MTCNN, InceptionResnetV1

#device = 'cuda:0' if torch.cuda.is_available() else 'cpu' #checks if GPY is being used or the CPU
#print(f'Running on device: {device}')
#torch.cuda.get_device_name(0)

Have to run code below in the fisrt time running

In [ ]:
# Get all test videos
filenames = glob.glob('/kaggle/input/deepfake-detection-challenge/test_videos/*.mp4')

# Number of frames to sample (evenly spaced) from each video
n_frames = 10

with torch.no_grad():
    face_list_whole_data=[]
    for i, filename in enumerate(filenames[:5]):
        print(f'Processing {i+1:5n} of {len(filenames):5n} videos\r', end='')    
        try:
            # Create video reader and find length
            v_cap = cv2.VideoCapture(filename)
            v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))
            # Pick 'n_frames' evenly spaced frames to sample
            sample = np.linspace(0, v_len - 1, n_frames).round().astype(int)
            face_list_1video = []
            for j in range(v_len):
                if j in sample:
                    success, vframe = v_cap.read()
                    vframe = cv2.cvtColor(vframe, cv2.COLOR_BGR2RGB)
                    face_locations = face_recognition.face_locations(vframe)
                    face_list_1frame=[]
                    for face_location in face_locations:
                    # Print the location of each face in this image
                        top, right, bottom, left = face_location
                    #print("A face is located at pixel location Top: {}, Left: {}, Bottom: {}, Right: {}".format(top, left, bottom, right))
                    # Access the actual face itself:
                        face_image = vframe[top:bottom, left:right]
                        res = cv2.resize(face_image, dsize=(256, 256), interpolation=cv2.INTER_CUBIC)
                        face_list_1frame.append(res/255)
                    face_list_1video.append(face_list_1frame)
            face_list_whole_data.append(face_list_1video)
        except KeyboardInterrupt:
            raise Exception("Stopped.")

****Show one face in a selected frame

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(5, 5))
plt.grid(False)
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
ax.imshow(face_list_whole_data[1][1][0])


In [ ]:
all_prob_list=[0]*len(face_list_whole_data)
for i, video in enumerate(face_list_whole_data):
    prob_list=[]
    for j, frame in enumerate(video):
        img_array=np.array(frame)
        probabilistic_predictions = MesoNet_classifier.predict(img_array)
        prob_list.append(probabilistic_predictions)
    all_prob_list[i]=prob_list
        #predictions = [num_to_label[round(x[0])] for x in probabilistic_predictions]
        #print(predictions)

In [ ]:
all_prob_list

In [ ]:
bias = -0.4
weight = 0.068235746

submission = []
subm_prob=[]
for filename, prob in zip(filenames, all_prob_list):
    if prob is not None and len(prob) == 10:
        indiv_prob=[]
        for i in prob:
            #p = 1 / (1 + np.exp(-(bias + (weight * i).sum())))
            #indiv_prob.append(p)
            indiv_prob.append(i)
    subm_prob.append(indiv_prob)
    #else: prob = 0.5
    submission.append([os.path.basename(filename), sum(indiv_prob)/len(indiv_prob)])
        

In [ ]:
submission

In [ ]:
num_to_label = {1:"real", 0:"fake"}